# Generate Data

In [2]:
from itertools import combinations

import pandas as pd
import json
import generate_data as gd
import numpy as np
import psycopg2
import json 
import pandas as pd
import generate_data as gd
import sqlalchemy as sa
import os
import ast, itertools
from sklearn.feature_extraction.text import TfidfVectorizer
import re
password = os.environ['ML_POSTGRESS_URL'].split(':')[2].split("@")[0]
# host = os.environ['ML_POSTGRESS_HOST']
host = "2m43izca46y.db.cloud.edu.au"
database = "simulation"
engine = sa.create_engine( f"postgresql://max:{password}@{host}" + f"/{database}")
conn = psycopg2.connect(host=host, dbname=database, user='max', password=password)
# Import train test split 

# Import the train test split function from sci-kit
from sklearn.model_selection import train_test_split
# from sklearn. import train_test_split
# all_info = pd.read_clipboard()
# all_info.to_excel("data/general.xlsx", index=False)
# all_info = pd.read_excel("data/general.xlsx")

sim_id = 3891194763 # 5 classes
sim_id = 1278467595 # 4 classes
sim_id = 2330035447 
sim_id = 412560783
sim_id = 2925821139
sim_id = 623986934 # 6 classes
sim_id = 53548897 # 4 classes
sim_id = 3840264976 # 7 classes

sim_id = 3361664654
sim_id = 466052150
sim_id = 2741437726
sim_id = 2741437726

sim_id = 1871628256


sim_id = 832107684 # 12 classes
sim_id = 2078238514 # 13 classes
sim_id = 3074882167 # 14 classes
sim_id = 2276442825 # 16 classes

sim_id = 1801022628 # 20 classes
sim_id = 1871628256 # 15 classes
sim_id = 2741437726 # 11 classes
sim_id = 832107684 # 12 classes

sim_id = 4265997001 # 8 classes

sim_id = 1824777786 # 19 classes
sim_id = 726862791 # 21 classes
sim_id = 1121062968 # 20 classes
sim_id = 979389019 # 19 classes
sim_id = 1871628256 # 15 classes
sim_id = 37326262 # 10 classes
sim_id = 774609303 # 5 classes
sim_id = 3339144425 # 6 classes
sim_id = 2969394773 # 7 classes

run_name = "412560783"

sql_query = f"""
SELECT 
    *
FROM 
    nd_simulations_dictionary sims
WHERE
    simulation_id = '{sim_id}';
"""
df_all= pd.read_sql(sql_query, conn)
conn.close()

# single_info = all_info[all_info['simulation_id'] == sim_id]

single_info = df_all[df_all['simulation_id'] == sim_id]
print(single_info)
for index, row in single_info.iterrows():
    dataset_name = row['name']
    simulation_id = row['simulation_id']
    seed = int(float(row['seed']))
    n_samples = int(float(row['n_samples']))
    n_classes = int(row['n_classes'])
    n_features = int(row['n_features'])
    # cov_beta = json.loads(row['covariates'])
    # intercepts = json.loads(row['intercept'])
    x_hash = int(row['x_hash'])
    y_hash = int(row['y_hash'])
    full_dataset_hash = int(row['full_dataset_hash'])
    extra_conditions = str(row['extra_condition'])
    # given_best_tree = row['selected_tree']
    cov = ast.literal_eval(row['covariates'])
    nd_structure = ast.literal_eval(row['generated_nd'])
    nd_params = ast.literal_eval(row['nd_params'])
    print(nd_params)

    best_prev_tree = tuple(gd.bfs_splits(tuple(nd_structure)))

    # X, y, dist_types, proba, freq = gd.generate_mlr_data(seed, n_features, n_samples, n_classes, cov_beta, intercepts, extra_conditions=extra_conditions)
    X, y, dist_types, freq = gd.generate_nd_data(seed, n_features, n_samples, n_classes, cov, nd_structure, nd_params=nd_params, extra_conditions=extra_conditions)
    # gd.check_hashes(X, y, x_hash, y_hash, full_dataset_hash)
    df = pd.DataFrame(X, columns=[f"p{i+1}" for i in range(n_features)])
    # df['Y'] = y
    X, X_test, y, y_test = train_test_split(df, y, test_size=0.2, random_state=seed)
    categories = tuple(np.unique(y))
    print(categories)

def v2_defined_all_trees(n: int):
    """Return a list of all ND trees over labels 0..n-1 (each tree is a tuple of splits)."""
    return list(_gen(tuple(range(n))))

def _gen(labels):
    labels = tuple(sorted(labels))
    if len(labels) <= 1:
        yield ()
        return
    s = labels[0]  # canonical rule: the smallest label stays on the 'left' side
    for r in range(1, len(labels)):
        for rest in combinations(labels[1:], r - 1):
            left  = tuple(sorted((s,) + rest))
            right = tuple(x for x in labels if x not in left)
            # normalize pair so children are unordered: larger side first, then lexicographic
            pair  = tuple(sorted((left, right), key=lambda t: (-len(t), t)))

            L = ((),) if len(left)  == 1 else tuple(_gen(left))
            R = ((),) if len(right) == 1 else tuple(_gen(right))

            for lt in L:
                for rt in R:
                    # pre-order DFS (shortest code). Swap to (lt+rt+(pair,)) if you want post-order.
                    yield (pair,) + lt + rt

# trees_defined = v2_defined_all_trees(len(categories))
# trees_defined
# len(set(trees_defined))
# tree_dict = {"tree": trees_defined}
# df_trees = pd.DataFrame(tree_dict)

/var/folders/h2/7j0s980d2w7951jcy8zc25ww08y435/T/ipykernel_52096/3242137034.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_all= pd.read_sql(sql_query, conn)


   simulation_id                                               name  seed  \
0   2.969395e+09  nd_class=7_features=8_seed=42_samples=100000_[...  42.0   

   n_classes  n_features  n_samples  \
0        7.0         8.0   100000.0   

                                  distribution_types  \
0  ['Col1: normal(2.18, 2.21)', 'Col2: normal_bin...   

                                        generated_nd  \
0  [((0, 1, 2, 3, 4, 5), (6,)), ((0, 1, 2, 3, 4),...   

                                          covariates  \
0  [['0.3', '-0.1', '-1.4', '1', '-0.6', '0.6', '...   

                                          class_freq  \
0  [0.16625 0.16947 0.16061 0.12791 0.11094 0.136...   

                                              nd_dot  \
0  digraph ND {\n  rankdir=TB;\n  N0 [shape=ellip...   

                                           nd_params        x_hash  \
0  {((0, 1, 2, 3, 4, 5), (6,)): ['0.3', '-0.1', '...  2.916425e+09   

        y_hash  full_dataset_hash extra_condition notes  
0 

In [4]:
# Set up 
import math
from scipy.special import factorial2
import random
from numpy.random import Generator, PCG64
# General imports
import numpy as np
from sklearn.linear_model import LogisticRegression
from numpy.random import Generator, PCG64
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.sm_exceptions import PerfectSeparationError, ConvergenceWarning
import warnings
from statsmodels.base.model import GenericLikelihoodModel
import numpy as np
import arviz as az
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
from bokeh.palettes import Category10

import numpy as np
from bokeh.plotting import figure, show, save, output_file
from bokeh.layouts import column as bcol, row as brow   # <-- alias to avoid shadowing
from bokeh.models import ColumnDataSource, Div
from bokeh.palettes import Category10
from graphviz import Source

def viz_mh_trace(traces, burn=0, top_k=20, html_path=None, best_known=None, loglik_fn=None, notes=""):
    try: L = min(len(t) for t in traces); arr = np.stack([np.asarray(t[:L], object) for t in traces], 0)
    except Exception: arr = np.asarray(traces, object)[None, :]

    flat = arr.ravel()
    uniq_trees, inv = np.unique(flat, return_inverse=True)
    arr_id = inv.reshape(arr.shape)

    accs = [(tr[1:] != tr[:-1]).mean() if tr.size > 1 else np.nan for tr in arr_id]
    overall = float(np.nanmean(accs)) if len(accs) else np.nan

    post = arr_id[:, burn:].ravel()
    uniq, counts = np.unique(post, return_counts=True)
    order = counts.argsort()[::-1][:top_k]
    top_ids, probs = uniq[order], counts[order] / counts.sum()
    labels = [str(int(i)) for i in top_ids]
    top_trees = [uniq_trees[int(i)] for i in top_ids]

    _ll = (lambda T: float(loglik_fn(T))) if loglik_fn else (lambda T: None)
    bk_ll = _ll(best_known) if (best_known is not None) else None
    top_lls = [_ll(T) for T in top_trees]
    if loglik_fn and any(ll is not None for ll in top_lls):
        j = int(np.nanargmax(top_lls)); best_found_id, best_found_ll, best_found_tree = int(top_ids[j]), top_lls[j], top_trees[j]
    else:
        best_found_id, best_found_ll, best_found_tree = int(top_ids[0]), None, top_trees[0]

    pal = Category10[10]

    p1 = figure(title="Trace (tree id)", width=720, height=240,
                tools="xpan,xwheel_zoom,reset,hover",
                tooltips=[("iter","@it"),("id","@idx"),("chain","@ch")])
    for c, tr in enumerate(arr_id):
        it = np.arange(tr.size); src = ColumnDataSource(dict(it=it, idx=tr, ch=[str(c)]*tr.size))
        p1.line('it','idx', source=src, alpha=0.85, line_width=2, color=pal[c%10])
        p1.circle('it','idx', source=src, size=3, alpha=0.6, color=pal[c%10])

    def _acf(x, L=50):
        x = np.asarray(x, float); x -= x.mean(); den = (x*x).sum() or 1.0
        K = min(L, max(1, x.size-1)); return np.array([1.0] + [float(np.dot(x[:-k], x[k:]) / den) for k in range(1, K+1)])
    p2 = figure(title="Autocorrelation", width=720, height=220, x_axis_label="lag",
                tools="xpan,xwheel_zoom,reset,hover", tooltips=[("lag","@lag"),("acf","@acf{0.000}")])
    for c, tr in enumerate(arr_id):
        ac = _acf(tr); src = ColumnDataSource(dict(lag=np.arange(ac.size), acf=ac))
        p2.line('lag','acf', source=src, line_width=2, alpha=0.9, color=pal[c%10])

    src2 = ColumnDataSource(dict(idx=labels, p=probs))
    p3 = figure(x_range=labels, title=f"Posterior mass (top {top_k})", width=720, height=260,
                tools="hover", tooltips=[("id","@idx"),("mass","@p{0.000}")])
    p3.vbar(x='idx', top='p', source=src2, width=0.9)

    ac_labels = [f"ch{c}" for c in range(arr_id.shape[0])]
    src3 = ColumnDataSource(dict(ch=ac_labels, a=[float(a) for a in accs]))
    p4 = figure(x_range=ac_labels, title=f"Acceptance (overall {overall:.3f})", width=720, height=200,
                tools="hover", tooltips=[("chain","@ch"),("acc","@a{0.000}")])
    p4.vbar(x='ch', top='a', source=src3, width=0.7)

    def _fmt_tree(t): s = str(t); return s if len(s) <= 800 else s[:797] + "…"
    def _fmt_tree_lines(t):
        try: return "\n".join(str((tuple(L), tuple(R))) for (L,R) in t)
        except Exception: return _fmt_tree(t)

    def _dot_svg(T):
        try:
            by = {frozenset((*L,*R)):(L,R) for (L,R) in T}
            root = set().union(*[set(L)|set(R) for (L,R) in T])
            dot = ["digraph ND{rankdir=TB;node[shape=box,fontsize=9];"]; seen=set()
            def rec(U):
                Ufs=frozenset(U)
                if Ufs in seen: return
                seen.add(Ufs)
                name="n_"+"_".join(map(str,sorted(U))); lab="{"+",".join(map(str,sorted(U)))+"}"
                dot.append(f'{name}[label="{lab}"];')
                if len(U)<=1: return
                L,R=by[frozenset(U)]
                for V in (set(L),set(R)):
                    v="n_"+"_".join(map(str,sorted(V))); dot.append(f"{name}->{v};"); rec(V)
            rec(root); dot.append("}")
            return Source("\n".join(dot)).pipe(format="svg").decode("utf-8")
        except Exception:
            return "<!-- graphviz failed -->"

    bk_svg = _dot_svg(best_known) if best_known is not None else ""
    bf_svg = _dot_svg(best_found_tree) if best_found_tree is not None else ""

    bk_txt = "" if best_known is None else f"<h3>Best-known</h3><pre>{_fmt_tree_lines(best_known)}</pre><br>"
    bf_txt = f"<h3>Best-found (id {best_found_id}{'' if best_found_ll is None else f', ll {best_found_ll:.3f}'})</h3><pre>{_fmt_tree_lines(best_found_tree)}</pre>"

    tops = "".join(f"<li><b>id {int(i)}</b> (p={p:.3f}{'' if (top_lls[k] is None) else f', ll {top_lls[k]:.3f}'})<br><code>{_fmt_tree(top_trees[k])}</code></li>"
                   for k,(i,p) in enumerate(zip(top_ids, probs)))

    div_summary = Div(text=f"<h2>MH Summary</h2>{'' if bk_ll is None else f'<p><b>Best-known ll:</b> {bk_ll:.3f}</p>'}{notes}")
    div_best = Div(text=f"<table><tr>"
                        f"<td valign='top'>{bk_txt}{bk_svg}</td>"
                        f"<td style='width:20px'></td>"
                        f"<td valign='top'>{bf_txt}{bf_svg}</td>"
                        f"</tr></table>")
    div_list = Div(text=f"<h3>Top {top_k} trees</h3><ol>{tops}</ol>")

    layout = brow(bcol(p1, p2, p3, p4), bcol(div_summary, div_best, div_list))  # <-- use aliases here
    if html_path: output_file(html_path, title="MH Diagnostics"); save(layout)
    else: show(layout)

    return {
        "ids_map": {int(i): uniq_trees[int(i)] for i in top_ids},
        "top_ids": top_ids, "top_probs": probs, "top_trees": top_trees,
        "acc": accs, "overall": overall,
        "best_found_id": best_found_id, "best_found_ll": best_found_ll, "best_found_tree": best_found_tree,
        "best_known_ll": bk_ll
    }



def _canon_split(left_set, right_set):
    """Canonicalize a split (A,B) so (A,B)==(B,A) for hashing/caching."""
    A = tuple(sorted(left_set))
    B = tuple(sorted(right_set))
    return (A, B) if A <= B else (B, A)

def tree_signatures(tree):
    """
    Tree is a tuple/list of splits from defined_all_trees:
      ((A1,B1), (A2,B2), ..., (A_{n-1},B_{n-1}))
    Return a frozenset of canonicalized splits.
    """
    return frozenset(_canon_split(a, b) for (a, b) in tree)

T = (((0, 1), (2, 3)), ((0,), (1,)), ((2,), (3,)))
T = (((0, 1, 2), (3,)), ((0, 1), (2,)), ((0,), (1,)))
T = (((0, 1, 2, 3), (4,)), ((0, 1, 2), (3,)), ((0, 1), (2,)), ((0,), (1,)))

rng = Generator(PCG64(42))

from collections import deque

def bfs_splits(tree):
    # normalise each split: sort labels; bigger side left (tie → lexicographic)
    S = [ (tuple(sorted(L)), tuple(sorted(R))) for (L,R) in tree ]
    S = [ (max(a,b, key=lambda t:(len(t), t)), min(a,b, key=lambda t:(len(t), t))) for a,b in S ]

    # map subset → split; find root label set
    by = { frozenset((*L, *R)): (L, R) for (L, R) in S }
    root = set().union(*[ set(L)|set(R) for (L,R) in S ])

    # BFS order
    out, q = [], deque([root])
    while q:
        U = q.popleft()
        if len(U) <= 1: 
            continue
        L, R = by[frozenset(U)]
        out.append((L, R))
        q.append(set(L)); q.append(set(R))
    return tuple(out)


def local_swap(T, rg):
    all_splits = list(T)
    eligible_splits = [x for x in all_splits if len(x[0]) > 1 or len(x[1]) > 1]
    # print(eligible_splits)
    node_to_perform_on = random.sample(eligible_splits, 1)[0]
    # local_item_swap
    L = list(node_to_perform_on[0])
    R = list(node_to_perform_on[1])
    left_swap = random.sample(L, 1)[0]
    right_swap = random.sample(R, 1)[0]
    # print(f"Swapping {left_swap} with {right_swap}")
    # print(f"Original node: {node_to_perform_on}")
    L.remove(left_swap)
    R.remove(right_swap)
    new_L = L + [right_swap]
    new_R = R + [left_swap]
    new_node = (tuple(new_L), tuple(new_R))

    # Now cascade the changes down the tree
    T = list(T)
    S   = set(new_node[0]) | set(new_node[1])
    U = [set(L)|set(R) for (L,R) in T]
    desc_idx = [k for k in range(0, len(T)) if U[k].issubset(S)]

    for k in desc_idx:
        L, R = T[k]
        A = np.concatenate([np.asarray(L), np.asarray(R)])  
        ma, mb = (A == left_swap), (A == right_swap)
        if ma.any() or mb.any():
            A[ma], A[mb] = right_swap, left_swap
            nL = len(L)
            c = [sorted(A[:nL]), sorted(A[nL:])]
            c.sort(key=len, reverse=True)
            l_sub = c[0]
            r_sub = c[1]
            T[k] = (tuple(l_sub), tuple(r_sub))

    # T = gd.breadth_first_splits(tuple(T))

    return bfs_splits(T), 0.0

def entire_node_swap(T, rg):
    total_possible = 0
    # all_splits = gd.breadth_first_splits(T)
    all_splits = list(T)
    eligible_splits = [x for x in all_splits if len(x[0]) > 1 or len(x[1]) > 1]
    node_to_perform_on = random.sample(eligible_splits, 1)[0]
    total_possible += math.comb(len(eligible_splits), 1)
    node_choices = len(eligible_splits)
    S = set(node_to_perform_on[0]) | set(node_to_perform_on[1])
    original_nodes = [ (L,R) for (L,R) in all_splits if (set(L)|set(R)).issubset(S) and (len(L)+len(R) >= 2) ]

    # entire node swap
    L = list(node_to_perform_on[0])
    R = list(node_to_perform_on[1])
    all_classes = set(L + R)
    number_of_classes = len(all_classes)
    left_split_size = random.choice(range(1,number_of_classes))
    # total_possible += factorial2(2*number_of_classes - 3)
    new_L = sorted(random.sample(list(all_classes), left_split_size))
    new_R = sorted(list(set(all_classes) - set(new_L)))
    new_node = (tuple(new_L), tuple(new_R))
    T = list(T)
    T.remove(node_to_perform_on)
    T.append(new_node)
    g_top = 1.0 / ((number_of_classes - 1) * math.comb(number_of_classes, left_split_size))

    # Now cascade the changes down the tree
    T = list(T)
    S   = set(new_node[0]) | set(new_node[1])

    rm = {i for i,(L,R) in enumerate(T) if (set(L)|set(R)).issubset(S)}
    T  = [x for i,x in enumerate(T) if i not in rm]

    def grow(classes):
        classes = list(classes)
        n = len(classes)
        if n <= 1:
            return [], 1.0
        l_split_size = random.choice(range(1,n))                  
        A = set(random.sample(classes, l_split_size))
        B = set(classes) - A
        c = [A, B]
        c.sort(key=len, reverse=True)
        A = c[0]
        B = c[1]
        left_splits,  pL = grow(A)
        right_splits, pR = grow(B)
        node = (tuple(sorted(A)), tuple(sorted(B)))
        return [node] + left_splits + right_splits, (1.0 / ((n - 1) * math.comb(n, l_split_size))) * pL * pR
    
    left_desc,  p_left  = grow(new_node[0])
    right_desc, p_right = grow(new_node[1])
    T += [new_node] + left_desc + right_desc
    # T = gd.breadth_first_splits(tuple(T))

    # print(f"Swapping entire node {node_to_perform_on} with {new_node}")
    # 5) full probability g(T*|T)
    E_star = sum(1 for L,R in T if len(L)>1 or len(R)>1)
    g_rev = 1.0 / E_star
    for (L,R) in original_nodes:              
        mm, kk = len(L)+len(R), len(L)       
        g_rev *= 1.0 / ((mm - 1) * math.comb(mm, kk))

    g_fwd = (1.0 / node_choices) * g_top * p_left * p_right
    return bfs_splits(T), g_fwd, g_rev

print(local_swap(T, rng))
print(entire_node_swap(T, rng))

((((0, 1, 2, 3), (4,)), ((0, 1, 2), (3,)), ((1, 2), (0,)), ((2,), (1,))), 0.0)
((((0, 1, 2, 3), (4,)), ((0, 1, 2), (3,)), ((0, 2), (1,)), ((2,), (0,))), 0.027777777777777776, 0.027777777777777776)


# 1 - Particle Filtering Basic

Just a first try at particle filtering

In [3]:
# pip install particles
import numpy as np, random, math, particles
from numpy.random import Generator, PCG64
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

def init_tree_n(n_classes, rng):

    def grow(classes):
        classes = list(classes)
        n = len(classes)
        if n <= 1:
            return []
        l_split_size = random.choice(range(1,n))                  
        A = set(random.sample(classes, l_split_size))
        B = set(classes) - A
        left_splits = grow(A)
        right_splits= grow(B)
        node = (tuple(sorted(A)), tuple(sorted(B)))
        return [node] + left_splits + right_splits

    return grow(range(0, n_classes))

def split_loglik_star_sm(X, y, split, maxiter=200, eps=1e-12):
    """
    ℓ*(split) using statsmodels Logit (unpenalized MLE).
    Returns the node log-likelihood at the MLE (float).
    Degenerate targets (all 0/1) → return 0.0 (supremum in the limit).
    """
    A, B = split
    mask = np.isin(y, A + B)
    if not np.any(mask):
        return 0.0
    s = np.isin(y[mask], B).astype(int)

    if s.min() == s.max():
        return 0.0
    Xsub = sm.add_constant(X[mask], has_constant="add")
    try: 
        # res = sm.Logit(s, Xsub).fit(disp=0, maxiter=maxiter, method="newton", tol=1e-6)
        # return float(res.llf)
        # res = sm.Logit(s, Xsub).fit(disp=0, maxiter=maxiter, method="lbfgs", tol=1e-6)
        # Direct log-likelihood at the MLE:
        lr = LogisticRegression(penalty=None, solver="newton-cholesky", max_iter=maxiter)
        # lr = LogisticRegression(penalty=None, solver="newton-cholesky", max_iter=maxiter)
        lr.fit(X[mask], s)
        p = lr.predict_proba(X[mask])[:, 1]
        return float(-log_loss(s, p, normalize=False))
    except PerfectSeparationError:
        print("Perfect Separation Error.")
        return 0.0
    except Exception as e:
        raise e
        # Any numerical prob: treat as saturated/flat
        print("Failed to do normal so we are fitting regularized.")
        p = np.clip(sm.Logit(s, Xsub).fit_regularized(alpha=1e-6, L1_wt=0.0, maxiter=maxiter).predict(), eps, 1-eps)
        return float((s*np.log(p) + (1-s)*np.log(1-p)).sum())
    
import numpy as np, random, particles
from numpy.random import Generator, PCG64
def _tree_ll_batch(Xb, yb, T):
    return sum(split_loglik_star_sm(Xb, yb, s) for s in tree_signatures(T))

_normT = lambda T: tuple((tuple(L), tuple(R)) for (L, R) in T)  # <-- ensure pure tuples

class NDFK(particles.FeynmanKac):
    def __init__(self, X, y, batches=10, lam=0.6, n_classes=None, seed=42):
        self.X = np.asarray(getattr(X,'values',X)); self.y = np.asarray(getattr(y,'values',y))
        self.lam = lam; self.rng = Generator(PCG64(seed)); random.seed(seed)
        idx = np.arange(len(self.y)); self.rng.shuffle(idx)
        sp = np.array_split(idx, batches) if isinstance(batches,int) else [np.asarray(b,int) for b in batches]
        self.splits = [np.asarray(b, int).ravel() for b in sp]
        self.T = len(self.splits); self.K = n_classes or int(np.unique(self.y).size)

    def M0(self, N):
        return np.array([_normT(bfs_splits(init_tree_n(self.K, self.rng))) for _ in range(N)], dtype=object)

    def M(self, t, xp):
        out = []
        for T in xp:
            T = _normT(T)  # <-- make tuples
            if self.rng.uniform() < self.lam:
                T, _   = local_swap(T, self.rng)
            else:
                T, *_  = entire_node_swap(T, self.rng)
            out.append(_normT(T))
        return np.asarray(out, dtype=object)

    def logG(self, t, xp, x):
        I = self.splits[t] 
        Xb, yb = self.X[I], self.y[I]
        return np.array([_tree_ll_batch(Xb, yb, T) for T in x], float)

# ---- quick run ----
fk = NDFK(X, y, batches=10, lam=0.6, seed=42)
pf = particles.SMC(fk=fk, N=256, resampling='systematic', ESSrmin=0.05, store_history=False)
pf.run()
order = np.argsort(-pf.W)
top_trees = [_normT(pf.X[i]) for i in order[:20]]
print(top_trees)

# ---- step-by-step ----
# fk = NDFK(X, y, batches=10, lam=0.6, seed=42)
# pf = particles.SMC(fk=fk, N=256, resampling='systematic', ESSrmin=0.5, store_history=False)
# pf.initialise()
# for _ in range(fk.T): pf.step()
# order = np.argsort(-pf.W); top_trees = [pf.X[i] for i in order[:20]]

print("Top pf tree is:", _normT(pf.X[order[0]]))
if bfs_splits(best_prev_tree) in top_trees:
    print("The best tree is found inside top trees!")
else:
    print("The best tree is NOT found by particle filtering.")
    print("Best tree is:", best_prev_tree)

NameError: name 'bfs_splits' is not defined

In [ ]:
# pip install particles
import numpy as np, random, math, particles
from numpy.random import Generator, PCG64
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

def init_tree_n(n_classes, rng):

    def grow(classes):
        classes = list(classes)
        n = len(classes)
        if n <= 1:
            return []
        l_split_size = random.choice(range(1,n))                  
        A = set(random.sample(classes, l_split_size))
        B = set(classes) - A
        left_splits = grow(A)
        right_splits= grow(B)
        node = (tuple(sorted(A)), tuple(sorted(B)))
        return [node] + left_splits + right_splits

    return grow(range(0, n_classes))

def split_loglik_star_sm(X, y, split, maxiter=200, eps=1e-12):
    """
    ℓ*(split) using statsmodels Logit (unpenalized MLE).
    Returns the node log-likelihood at the MLE (float).
    Degenerate targets (all 0/1) → return 0.0 (supremum in the limit).
    """
    A, B = split
    mask = np.isin(y, A + B)
    if not np.any(mask):
        return 0.0
    s = np.isin(y[mask], B).astype(int)

    if s.min() == s.max():
        return 0.0
    Xsub = sm.add_constant(X[mask], has_constant="add")
    try: 
        # res = sm.Logit(s, Xsub).fit(disp=0, maxiter=maxiter, method="newton", tol=1e-6)
        # return float(res.llf)
        # res = sm.Logit(s, Xsub).fit(disp=0, maxiter=maxiter, method="lbfgs", tol=1e-6)
        # Direct log-likelihood at the MLE:
        lr = LogisticRegression(penalty=None, solver="newton-cholesky", max_iter=maxiter)
        # lr = LogisticRegression(penalty=None, solver="newton-cholesky", max_iter=maxiter)
        lr.fit(X[mask], s)
        p = lr.predict_proba(X[mask])[:, 1]
        return float(-log_loss(s, p, normalize=False))
    except PerfectSeparationError:
        print("Perfect Separation Error.")
        return 0.0
    except Exception as e:
        raise e
        # Any numerical prob: treat as saturated/flat
        print("Failed to do normal so we are fitting regularized.")
        p = np.clip(sm.Logit(s, Xsub).fit_regularized(alpha=1e-6, L1_wt=0.0, maxiter=maxiter).predict(), eps, 1-eps)
        return float((s*np.log(p) + (1-s)*np.log(1-p)).sum())
    
import numpy as np, random, particles
from numpy.random import Generator, PCG64
def _tree_ll_batch(Xb, yb, T):
    return sum(split_loglik_star_sm(Xb, yb, s) for s in tree_signatures(T))

_normT = lambda T: tuple((tuple(L), tuple(R)) for (L, R) in T)  # <-- ensure pure tuples

class NDFK(particles.FeynmanKac):
    def __init__(self, X, y, batches=10, lam=0.6, n_classes=None, seed=42):
        self.X = np.asarray(getattr(X,'values',X)); self.y = np.asarray(getattr(y,'values',y))
        self.lam = lam; self.rng = Generator(PCG64(seed)); random.seed(seed)
        idx = np.arange(len(self.y)); self.rng.shuffle(idx)
        sp = np.array_split(idx, batches) if isinstance(batches,int) else [np.asarray(b,int) for b in batches]
        self.splits = [np.asarray(b, int).ravel() for b in sp]
        self.T = len(self.splits); self.K = n_classes or int(np.unique(self.y).size)

    def M0(self, N):
        return np.array([_normT(bfs_splits(init_tree_n(self.K, self.rng))) for _ in range(N)], dtype=object)

    def M(self, t, xp):
        out = []
        for T in xp:
            T = _normT(T)  # <-- make tuples
            if self.rng.uniform() < self.lam:
                T, _   = local_swap(T, self.rng)
            else:
                T, *_  = entire_node_swap(T, self.rng)
            out.append(_normT(T))
        return np.asarray(out, dtype=object)

    def logG(self, t, xp, x):
        I = self.splits[t] 
        Xb, yb = self.X[I], self.y[I]
        return np.array([_tree_ll_batch(Xb, yb, T) for T in x], float)

# ---- quick run ----
fk = NDFK(X, y, batches=10, lam=0.6, seed=42)
pf = particles.SMC(fk=fk, N=256, resampling='systematic', ESSrmin=0.5, store_history=False)
pf.run()
order = np.argsort(-pf.W)
top_trees = [_normT(pf.X[i]) for i in order[:20]]
print(top_trees)

# ---- step-by-step ----
# fk = NDFK(X, y, batches=10, lam=0.6, seed=42)
# pf = particles.SMC(fk=fk, N=256, resampling='systematic', ESSrmin=0.5, store_history=False)
# pf.initialise()
# for _ in range(fk.T): pf.step()
# order = np.argsort(-pf.W); top_trees = [pf.X[i] for i in order[:20]]

print("Top pf tree is:", _normT(pf.X[order[0]]))
if bfs_splits(best_prev_tree) in top_trees:
    print("The best tree is found inside top trees!")
else:
    print("The best tree is NOT found by particle filtering.")
    print("Best tree is:", best_prev_tree)

# 2 - Particle Filtering Build Up 

# Layer-by-Layer SMC for Nested Dichotomies (uniform proposal)

**Classes.** Let $\mathcal{C}=\{0,1,\ldots,K-1\}$.

**State at time $t$.** $x_t=(S_t,O_t)$ where  
- $S_t$ is a tuple of internal splits so far; each split is an unordered pair $(A,B)$ with $A\cap B=\varnothing$, $A\cup B=L$ for some leaf $L$.  
- $O_t$ is a tuple of open (unsplit) leaves; each leaf is a subset of $\mathcal{C}$.

**Horizon.** $T=K-1$ (add one split per step until all leaves are singletons).

**Initial state.** $S_0=\varnothing,\quad O_0=\{\mathcal{C}\}$.

**Proposal $M_t$ (structure only).**
1. Pick a leaf $L\in O_{t-1}$ uniformly.  
2. Sample a uniform **unordered**, non-trivial binary partition $(A,B)$ of $L$.  
3. Update

$S_t=S_{t-1}\cup\{(A,B)\},\qquad
O_t=\big(O_{t-1}\setminus\{L\}\big)\cup\{A:\lvert A\rvert>1\}\cup\{B:\lvert B\rvert>1\}.
$

**Weights (potential) $G_t$ with batch $I_t$.**  
Define

$\ell^*=\sum_{(A,B)\in S}\ell(A,B;X_I,y_I)$,

where $\ell^*$ is the node log-likelihood at the MLE (degenerate $0/1$ target $\Rightarrow 0$).  
Then
$\log G_t=\ell(S_t;I_t)-\ell(S_{t-1};I_t).$


**Resampling.** Standard SMC (e.g., systematic) with an ESS threshold.


In [5]:
from collections import defaultdict
import numpy as np
from bokeh.plotting import figure, show, save, output_file
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Div

def viz_smc_snapshot(
    pf,
    top_k=20,
    html_path=None,
    title="SMC Snapshot",
    history=None,   # dict from collect_history_simple
    X=None, y=None  # if provided, compute LL bars for final top trees
):
    # ----- final snapshot (posterior over structures) -----
    w = np.asarray(pf.W, float); w = w / (w.sum() if w.sum() > 0 else 1.0)

    def tree_key(nodes):  # permutation-invariant key
        return frozenset(nodes)

    mass_by_tree = defaultdict(float)
    for i, (nodes, _open) in enumerate(pf.X):
        mass_by_tree[tree_key(nodes)] += float(w[i])

    ranked = sorted(mass_by_tree.items(), key=lambda kv: kv[1], reverse=True)[:top_k]
    top_keys = [k for (k, m) in ranked]
    probs    = [m for (k, m) in ranked]
    # Make displayable tuple-of-splits for the top trees
    top_trees = [tuple(sorted(k)) for k in top_keys]
    labels    = [f"T{k}" for k in range(len(ranked))]

    def fmt_tree(tree):
        try:
            return "<br>".join(str((tuple(A), tuple(B))) for (A,B) in tree)
        except Exception:
            return str(tree)

    figs = []

    # Posterior mass (top-k)
    src1 = ColumnDataSource(dict(id=labels, p=probs))
    p1 = figure(x_range=labels, title=f"Posterior mass (top {len(labels)})", width=720, height=220,
                tools="hover", tooltips=[("id","@id"),("mass","@p{0.000}")])
    p1.vbar(x="id", top="p", source=src1, width=0.9)
    figs.append(p1)

    # Sorted particle weights (final)
    sw = np.sort(w)[::-1]
    src2 = ColumnDataSource(dict(rank=np.arange(sw.size), w=sw))
    p2 = figure(title="Sorted particle weights (final)", width=720, height=200,
                tools="xpan,xwheel_zoom,reset,hover",
                tooltips=[("rank","@rank"),("w","@w{0.0000}")])
    p2.line("rank","w", source=src2, line_width=2)
    p2.circle("rank","w", source=src2, size=3, alpha=0.7)
    figs.append(p2)

    # ----- history (if provided) -----
    history_div = None
    if history is not None and len(history.get("ESS", [])):
        ESS_ts  = np.asarray(history["ESS"], float)
        uniq_ts = np.asarray(history["uniq"], float)
        t = np.arange(1, ESS_ts.size + 1)
        src_ts = ColumnDataSource(dict(t=t, ESS=ESS_ts, uniq=uniq_ts))

        # ESS and #unique trees over time
        p3 = figure(title="ESS & #unique trees over time", width=720, height=200,
                    tools="xpan,xwheel_zoom,reset,hover",
                    tooltips=[("t","@t"),("ESS","@ESS{0.0}"),("#uniq","@uniq")])
        p3.line("t","ESS",  source=src_ts, line_width=2, legend_label="ESS")
        p3.circle("t","ESS", source=src_ts, size=3, alpha=0.7)
        p3.line("t","uniq", source=src_ts, line_width=2, line_dash="dashed", legend_label="#unique")
        p3.circle("t","uniq", source=src_ts, size=3, alpha=0.7)
        p3.legend.location = "top_right"
        figs.append(p3)

        # Mass over time for final top-3 trees
        X_t, W_t = history["X_t"], history["W_t"]
        top_show = top_keys[:min(3, len(top_keys))]
        data_m = {"t": list(range(1, len(X_t) + 1))}
        for k, key_ref in enumerate(top_show):
            m_ts = []
            for Xt, Wt in zip(X_t, W_t):
                by = defaultdict(float)
                for i, (nodes, _ol) in enumerate(Xt):
                    by[tree_key(nodes)] += float(Wt[i])
                m_ts.append(by.get(key_ref, 0.0))
            data_m[f"m{k}"] = m_ts
        src_m = ColumnDataSource(data_m)
        p4 = figure(title="Mass over time (final top trees, up to 3)", width=720, height=200,
                    tools="xpan,xwheel_zoom,reset,hover", tooltips=[("t","@t")])
        for k in range(len(top_show)):
            p4.line("t", f"m{k}", source=src_m, line_width=2, legend_label=labels[k])
        p4.legend.location = "top_right"
        figs.append(p4)

        # Modal tree per step (simple history list)
        items = []
        for ti, (Xt, Wt) in enumerate(zip(X_t, W_t), start=1):
            by = defaultdict(float)
            for i, (nodes, _ol) in enumerate(Xt):
                by[tree_key(nodes)] += float(Wt[i])
            if by:
                key_mode, pmode = max(by.items(), key=lambda kv: kv[1])
                Tmode = tuple(sorted(key_mode))
                items.append(f"<li>t={ti}: p={pmode:.3f} <code>{fmt_tree(Tmode)}</code></li>")
        history_div = Div(text=f"<h4>History: modal tree per step</h4><ol>{''.join(items)}</ol>")

    # ----- optional: bars of log-likelihood for final top trees -----
    best_ll_vals = None
    if (X is not None) and (y is not None) and len(top_trees):
        def tree_ll(tree):
            try: return float(sum(split_loglik_star_sm(X, y, s) for s in tree))
            except Exception: return np.nan
        best_ll_vals = [tree_ll(t) for t in top_trees]
        src_ll = ColumnDataSource(dict(id=labels, ll=best_ll_vals))
        pLL = figure(x_range=labels, title="Log-likelihood (final top trees)", width=720, height=200,
                     tools="hover", tooltips=[("id","@id"),("ll","@ll{0.000}")])
        pLL.vbar(x="id", top="ll", source=src_ll, width=0.9)
        figs.append(pLL)

    # ----- summary -----
    ESS_final = 1.0 / float((w**2).sum()) if w.size else 0.0
    lst = "".join(f"<li><b>{labels[i]}</b> (p={probs[i]:.3f}"
                  + ("" if (best_ll_vals is None) else f", ll={best_ll_vals[i]:.3f}")
                  + f")<br><code>{fmt_tree(top_trees[i])}</code></li>"
                  for i in range(len(labels)))
    div_summary = Div(text=f"<h3>{title}</h3>"
                           f"<p>Particles: {w.size} &nbsp; | &nbsp; ESS(final): {ESS_final:.1f}</p>"
                           f"<h4>Top trees</h4><ol>{lst}</ol>")

    layout = column(*figs, div_summary, *( [history_div] if history_div is not None else [] ))
    if html_path:
        output_file(html_path, title=title); save(layout)
    else:
        show(layout)

    return {"top_trees": top_trees, "top_probs": probs, "ESS": ESS_final, "ll_top": best_ll_vals}

def collect_history_simple(fk, N=256, resampling='systematic', ESSrmin=0.5):
    """
    Runs SMC step-by-step once to capture history for plotting.
    Returns: pf, history dict with ESS, uniq, X_t, W_t.
    """
    pf = particles.SMC(fk=fk, N=N, resampling=resampling, ESSrmin=ESSrmin, store_history=False)
    pf.initialise()
    ESS, uniq, X_t, W_t = [], [], [], []
    for _ in range(fk.T):
        pf.step()
        w = np.asarray(pf.W, float); w = w / (w.sum() if w.sum() > 0 else 1.0)
        ESS.append(1.0 / float((w**2).sum()))
        uniq.append(len({frozenset(s[0]) for s in pf.X}))  # permutation-invariant unique
        X_t.append(list(pf.X))       # states at time t: list of (nodes, open_leaves)
        W_t.append(w.copy())         # weights at time t
    return pf, {"ESS": np.array(ESS), "uniq": np.array(uniq), "X_t": X_t, "W_t": W_t}


In [ ]:
import numpy as np, particles
from numpy.random import Generator, PCG64

def _canon(A, B):
    A, B = tuple(sorted(A)), tuple(sorted(B))
    return (A, B) if A < B else (B, A)

def _uniform_partition(leaf, rng):
    # uniform over unordered non-trivial splits using Python's random
    number_of_classes = len(leaf)
    # left_split_size = random.choice(range(1, number_of_classes))
    left_split_size = rng.choice(range(1, number_of_classes))
    m = rng.choice(list(leaf), size=left_split_size, replace=False)
    new_L = sorted(list(m))
    new_R = sorted(list(set(leaf) - set(new_L)))
    return _canon(tuple(new_L), tuple(new_R))

def _ll_nodes(Xb, yb, nodes):
    return sum(split_loglik_star_sm(Xb, yb, s) for s in nodes)

def _close_pair_leaves(nodes, open_leaves):
    """
    Deterministically finish any open leaves of size 2:
    e.g. (3,4) -> add ((3,), (4,)) to nodes and remove the leaf from open_leaves.
    Returns (nodes_list, open_leaves_list).
    """
    nodes = list(nodes)
    ol = list(open_leaves)
    while True:
        idx2 = [i for i, L in enumerate(ol) if len(L) == 2]
        if not idx2:
            break
        for i in reversed(idx2):
            a, b = tuple(sorted(ol.pop(i)))
            nodes.append(_canon((a,), (b,)))
    return nodes, ol
    
class NDFK_LBL(particles.FeynmanKac):
    """Layer-by-layer SMC: one uniform structural split per step; weights = incremental LL."""
    def __init__(self, X, y, batches=10, n_classes=None, seed=42, rho=1.0):
        self.X = np.asarray(getattr(X, 'values', X))
        self.y = np.asarray(getattr(y, 'values', y))
        self.K = n_classes or int(np.unique(self.y).size)
        # self.T = max(0, self.K - 1)
        self.T = max(0, self.K *2)
        self.rng = Generator(PCG64(seed))
        idx = np.arange(len(self.y)); self.rng.shuffle(idx)
        self.batches = [np.asarray(b, int) for b in (np.array_split(idx, batches) if isinstance(batches, int) else batches)]
        self.B = len(self.batches)

        self.rho = rho
        self.rho_sched = np.linspace(0.00000001, 0.2, self.T) if self.T>0 else np.array([1.0])

    def M0(self, N): 
        root = tuple(range(self.K))
        x0 = ((), (root,))  
        return np.array([x0] * N, dtype=object)

    def M(self, t, xp):
        out = []
        for nodes, open_leaves in xp:
            nodes, ol = _close_pair_leaves(nodes, open_leaves)
            if ol:
                j = int(self.rng.integers(0, len(ol)))
                leaf = ol.pop(j)
                A, B = _uniform_partition(leaf, self.rng)
                nodes = tuple(list(nodes) + [(A, B)])
                if len(A) > 1: 
                    ol.append(A)
                if len(B) > 1: 
                    ol.append(B)
                ol = tuple(sorted(ol))
            else: 
                nodes, _   = local_swap(nodes, self.rng)
            nodes, ol = _close_pair_leaves(nodes, ol)
            out.append((nodes, ol))
        return np.asarray(out, dtype=object)

    def logG(self, t, xp, x):
        I = self.batches[t % self.B]
        Xb, yb = self.X[I], self.y[I]
        # if xp is None:
        #     prev_ll = np.zeros(len(x), dtype=float)
        # else:
        #     prev_ll = np.array([_ll_nodes(Xb, yb, n) for (n, _) in xp], float)
        curr_ll = np.array([_ll_nodes(Xb, yb, n) for (n, _) in x], float)
        rho_t = float(self.rho_sched[min(t, len(self.rho_sched)-1)])
        return rho_t * (curr_ll)
        # return self.rho * (curr_ll - prev_ll)


# ---- quick run  ----
fk = NDFK_LBL(X, y, batches=10, seed=43, rho=0.05)
# pf = particles.SMC(fk=fk, N=256, resampling='systematic', ESSrmin=0.1, store_history=False)
pf = particles.SMC(fk=fk, N=256, resampling='residual', ESSrmin=0.05, store_history=False)

# pf = particles.SMC(fk=fk, N=256, resampling='stratified', ESSrmin=0.9, store_history=False)

# pf = particles.SMC(fk=fk, N=384, resampling='residual', ESSrmin=0.25)

pf.run()
idx = np.argsort(pf.W)[::-1]
states = [pf.X[i] for i in idx]          
best_nodes, best_open = states[0]

# top_k node-sets (trees)
top_k = 20
top_states = states[:top_k]
top_trees = [bfs_splits(s[0]) for s in top_states]   

print("Top trees (first 3 shown):", top_trees[:3])
print("Best tree nodes:", best_nodes)

if bfs_splits(best_prev_tree) in top_trees:
    print("The best tree is found inside top trees!")
else:
    print("The best tree is NOT found by particle filtering.")
    print("Best tree is:", best_prev_tree)

# summary = viz_smc_snapshot(pf, top_k=20, html_path=None, title="ND-SMC")

Top trees (first 3 shown): [(((0, 1, 2, 3, 4, 6), (5,)), ((0, 1, 2, 3, 4), (6,)), ((0, 1, 2, 3), (4,)), ((0, 1, 2), (3,)), ((0, 1), (2,)), ((1,), (0,))), (((0, 1, 2, 3, 4, 6), (5,)), ((0, 1, 2, 3, 4), (6,)), ((0, 1, 2, 3), (4,)), ((0, 1, 2), (3,)), ((0, 1), (2,)), ((1,), (0,))), (((0, 1, 2, 3, 4, 6), (5,)), ((0, 1, 2, 3, 4), (6,)), ((0, 1, 2, 3), (4,)), ((0, 1, 2), (3,)), ((0, 1), (2,)), ((1,), (0,)))]
Best tree nodes: [((0, 1, 2, 3, 4, 6), (5,)), ((0, 1, 2, 3, 4), (6,)), ((0, 1, 2, 3), (4,)), ((0, 1, 2), (3,)), ((0, 1), (2,)), ((1,), (0,))]
The best tree is NOT found by particle filtering.
Best tree is: (((0, 1, 2, 3, 4, 5), (6,)), ((0, 1, 2, 3, 4), (5,)), ((0, 1, 2, 3), (4,)), ((0, 1, 2), (3,)), ((0, 1), (2,)), ((1,), (0,)))


In [29]:
# Build pf as usual
pf = particles.SMC(fk=fk, N=256, resampling='residual', ESSrmin=0.05, store_history=False)
# pf = particles.SMC(fk=fk, N=256, resampling='systematic', ESSrmin=0.4, store_history=False)
# pf = particles.SMC(fk=fk, N=384, resampling='residual', ESSrmin=0.25)

# Step and record
ESS, uniq, X_t, W_t = [], [], [], []
for _ in range(fk.T):
    pf.next()
    w = np.asarray(pf.W, float); 
    w /= (w.sum() or 1.0)
    ESS.append(1.0 / (w**2).sum())
    uniq.append(len({frozenset(s[0]) for s in pf.X}))  # permutation-invariant unique structures
    X_t.append(list(pf.X))                              # states this step
    W_t.append(w.copy())                                # weights this step

history = {"ESS": np.array(ESS), "uniq": np.array(uniq), "X_t": X_t, "W_t": W_t}

# Now viz (same function you already have)
viz_smc_snapshot(pf, top_k=20, title="ND-SMC (with history)", history=history, X=X, y=y)

# viz_smc_snapshot(pf, top_k=20, title="ND-SMC (with history)", history=hist, X=X, y=y)

{'top_trees': [(((0, 1), (2,)),
   ((0, 1, 2), (3,)),
   ((0, 1, 2, 3), (4,)),
   ((0, 1, 2, 3, 4), (5,)),
   ((0, 1, 2, 3, 4, 5), (6,)),
   ((1,), (0,))),
  (((0, 1, 2), (3,)),
   ((0, 1, 2, 3), (4,)),
   ((0, 1, 2, 3, 4), (6,)),
   ((0, 1, 2, 3, 4, 6), (5,)),
   ((1, 2), (0,)),
   ((2,), (1,))),
  (((0, 1), (2,)),
   ((0, 1, 2), (3,)),
   ((0, 1, 2, 3), (4,)),
   ((0, 1, 2, 3, 4), (6,)),
   ((0, 1, 2, 3, 4, 6), (5,)),
   ((1,), (0,))),
  (((0, 1, 2), (3,)),
   ((0, 1, 2, 3), (4,)),
   ((0, 1, 2, 3, 4), (5,)),
   ((0, 1, 2, 3, 4, 5), (6,)),
   ((1, 2), (0,)),
   ((2,), (1,))),
  (((0, 1, 2), (3,)),
   ((0, 1, 2, 3), (5,)),
   ((0, 1, 2, 3, 4, 5), (6,)),
   ((0, 1, 2, 3, 5), (4,)),
   ((1, 2), (0,)),
   ((2,), (1,))),
  (((0, 1, 2, 3), (4,)),
   ((0, 1, 2, 3, 4), (5,)),
   ((0, 1, 2, 3, 4, 5), (6,)),
   ((1, 2), (3,)),
   ((1, 2, 3), (0,)),
   ((2,), (1,))),
  (((0, 1), (2,)),
   ((0, 1, 2), (3,)),
   ((0, 1, 2, 3), (6,)),
   ((0, 1, 2, 3, 5, 6), (4,)),
   ((0, 1, 2, 3, 6), (5,)),
   (